In [1]:
#from textblob import TextBlob
#from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random
import os
import re
import pandas as pd

In [12]:
moviedataset = pd.read_csv('corpus/moviedata.csv')
moviedataset = moviedataset.apply(lambda x: x.astype(str).str.lower())
moviedataset[['numberOfPeople']] = moviedataset[['numberOfPeople']].apply(pd.to_numeric)


restaurantDataset = pd.read_csv('corpus/restaurantBooking.csv')
restaurantDataset = restaurantDataset.apply(lambda x: x.astype(str).str.lower())
restaurantDataset[['numberOfPeople']] = restaurantDataset[['numberOfPeople']].apply(pd.to_numeric)
print(moviedataset)
print(restaurantDataset)

                    mname    genre             actor  seattype  \
0                 karwaan    drama        irfan khan  platinum   
1                 karwaan    drama        irfan khan    corner   
2                 karwaan    drama        irfan khan    middle   
3   happy fir bhaag jaegi   comedy    sonakshi sinha    middle   
4                  dabang   comedy    sonakshi sinha    middle   
5   happy fir bhaag jaegi   comedy    sonakshi sinha    corner   
6   happy fir bhaag jaegi   comedy    sonakshi sinha     front   
7   happy fir bhaag jaegi   comedy    sonakshi sinha  platinum   
8            harry potter  fantasy  daniel radcliffe  platinum   
9            harry potter  fantasy       emma watson  platinum   
10           harry potter  fantasy  daniel radcliffe     front   
11           harry potter  fantasy       emma watson     front   
12           harry potter  fantasy  daniel radcliffe    middle   
13           harry potter  fantasy  daniel radcliffe    middle   

         

In [13]:
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''

    context = IntentComplete()
    return 'action: ' + current_intent.action, context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    #print(attributes)
    for para in current_intent.params:
        #print(para.name)
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                if para.name=='mname':
                    context = MovieName()
                elif para.name == 'mlocation':
                    context = MovieLocation()
                elif para.name == 'language':
                    context = Language()
                elif para.name == 'genre':
                    context = Genre()
                elif para.name == 'actor':
                    context = Actor()
                elif para.name == 'theatre':
                    context = Theatre()
                elif para.name == 'time':
                    context = Time()
                elif para.name == 'date':
                    context = Date()
                elif para.name == 'numberOfPeople':
                    context = NumberOfPeople()
                elif para.name == 'seattype':
                    context = SeatType()
                
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        #print(intent)
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    #Scoring Algorithm, can be changed.
    scores = ngrammatch(clean_input)
    
    #choosing here the intent with the highest score
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    print(scores)
    if(current_intent==None):
        if(clean_input=="movie"):
            return loadIntent('params/newparams.cfg', 'movieChoice')
        elif(clean_input=="restaurant"):
            return loadIntent('params/newparams.cfg', 'BookRestaurant')
        else:
            return loadIntent('params/newparams.cfg',scores[-1][0])
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil, encoding='windows-1252').readlines()
            for i, line in enumerate(lines):
                line = line.lower()
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)

        #Extract entity and update it in attributes dict
        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        for entity in entities:
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)

        #Example of where the context is being used to do conditional branching.
        print(attributes)
        #if 'mname' in attributes  or (context.name=='MovieBooking_moviename' and context.active):
        #    match = attributes['mname']
        #    context.active = False
        return attributes, uinput

In [29]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        #constructing query to run on the data set
        qry = ' and '.join(["{} == '{}'".format(k,v) for k,v in self.attributes.items() if k !='numberOfPeople' and k!='greeting'])
        
        if 'numberOfPeople' in self.attributes:
            if not qry:
                qry = 'numberOfPeople >='+ convertToNumber(self.attributes['numberOfPeople'])
            else:
                qry = qry + ' and numberOfPeople >='+ convertToNumber(self.attributes['numberOfPeople'])
        if self.current_intent.name =='movieChoice':
            isDFEmpty, unique_attributes = add_unique_attributes(moviedataset, qry)
        elif self.current_intent.name =='BookRestaurant':
            isDFEmpty, unique_attributes = add_unique_attributes(restaurantDataset, qry)

        if isDFEmpty == True and self.current_intent.name =='movieChoice':
            prompt = 'No tickets available for '+ str(self.attributes)
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
            return prompt
        if isDFEmpty == True and self.current_intent.name =='BookRestaurant':
            prompt = 'No restaurant table available for '+ str(self.attributes)
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
            return prompt
        self.attributes.update(unique_attributes)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)
        #prompt being None means all parameters satisfied, perform the intent action
        
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
                prompt = prompt + '\n'+ str(unique_attributes) + '\n\n'+ 'Thank you'
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        elif self.context.name == 'MovieBooking_moviename':
            prompt = prompt +  '\n'+ str(moviedataset.query(qry)) 
        elif self.context.name == 'RestaurantBooking_restaurantNames':
            prompt = prompt +  '\n'+ str(restaurantDataset.query(qry)) 

            
        
        return prompt

def add_unique_attributes(df, qry):
    y = {}
    #print(qry)
    if not qry:
        return False, y
    subsetDF = df.query(qry)
    print(subsetDF)
    if subsetDF.empty:
        return True, y
    
    for col in subsetDF:
        if col == 'numberOfPeople':
            continue
        unique_vals = subsetDF[col].unique()
        if len(unique_vals) ==1:
            y[col] = unique_vals[0]
    return False, y

def convertToNumber(numberword):
    if numberword =='one':
        return '1'
    elif numberword == 'two':
        return '2'
    elif numberword == 'three':
        return '3'
    elif numberword == 'four':
        return '4'
    elif numberword == 'five':
        return '5'
    elif numberword == 'six':
        return '6'
    elif numberword == 'seven':
        return '7'
    elif numberword == 'eight':
        return '8'
    elif numberword == 'nine':
        return '9'
    elif numberword == 'ten':
        return '10'
    
    

In [32]:
session = Session()

print ('BOT: Hi! How may I assist you?')

while True:
    
    inp = input('User: ')
    print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
User: book me a table for two at mainland china for today noon
{'time': 'noon', 'numberOfPeople': 'two', 'date': 'today', 'name': 'mainland china'}
[('movieChoice', 0.08090619502686254), ('BookRestaurant', 0.171271597134429)]
Empty DataFrame
Columns: [name, cuisine, location, costtype, numberOfPeople, date, time]
Index: []
BOT: No restaurant table available for {'time': 'noon', 'numberOfPeople': 'two', 'date': 'today', 'name': 'mainland china'}
User: book me a table for two at mainland china for today night
{'time': 'nigh', 'numberOfPeople': 'two', 'date': 'today', 'name': 'mainland china'}
[('movieChoice', 0.07257167308515192), ('BookRestaurant', 0.15357300884955752)]
Empty DataFrame
Columns: [name, cuisine, location, costtype, numberOfPeople, date, time]
Index: []
BOT: No restaurant table available for {'time': 'nigh', 'numberOfPeople': 'two', 'date': 'today', 'name': 'mainland china'}
User: book me a table for two at mainland china for today evening
{'

KeyboardInterrupt: 